In [1]:
import requests
import ipykernel
import re
from notebook.notebookapp import list_running_servers
from notebook import notebookapp

from matplotlib import pyplot as plt
servers = list(notebookapp.list_running_servers())

TOKEN = servers[0]['token']

base_url = next(list_running_servers())['url']
r = requests.get(
    url=base_url + 'api/sessions',
    headers={'Authorization': 'token {}'.format(TOKEN),})

r.raise_for_status()
response = r.json()

kernel_id = re.search(
    'kernel-(.*).json',
    ipykernel.connect.get_connection_file()
).group(1)

NOTEBOOK_PATH = {
    r['kernel']['id']: r['notebook']['path']
    for r in response
}[kernel_id]
print(NOTEBOOK_PATH)

aposentadoria-ouro-comparacao/xgb.ipynb


In [64]:
import os
import glob
import pickle
from itertools import chain
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import fitz
from dodfminer.extract.polished.acts.aposentadoria import Retirements
import unidecode

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
import xgboost as xgb

In [3]:
%%time
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model', xgb.XGBClassifier(
        objective='multi:softprob',
        random_state=0,
        num_class=2
    ))
])

CPU times: user 979 µs, sys: 179 µs, total: 1.16 ms
Wall time: 1.18 ms


In [14]:
pwd

'/home/leomaffei/Desktop/projeto-ia-unb/experiments/members/maffei/aposentadoria-ouro-comparacao'

In [53]:
df_train_block = pd.read_csv('xgb_train.csv', sep='®', engine='python',
                       index_col=False)

df_test_block = pd.read_csv('xgb_test.csv', sep='®', engine='python',
                     index_col=False)

df_blocos = pd.concat([df_train_block, df_test_block])
df_blocos['len'] = df_blocos.text.map(len)

df_ouro = pd.read_csv('labeled.csv', sep=',', engine='python')
df_ouro = df_ouro[df_ouro.text.notna()]

df_ouro['len'] = df_ouro.text.map(len)

del df_train_block, df_test_block
df_blocos.shape, df_ouro.shape

((109305, 12), (5169, 14))

In [21]:
df_train.head(1)

,DATA_DODF,ATO,EMPRESA_ATO,COD_MATRICULA_ATO,NOME_ATO,CARGO,CLASSE,PADRAO,QUADRO,PROCESSO,FUND_LEGAL,text,labels
0,01/01/2019,CONCEDER APOSENTADORIA,"SECRETARIA DE ESTADO DE PLANEJAMENTO, ORCAMENT...",00317276,MARIA DE JESUS BARBOSA SANTOS,ANALISTA EM POLITICAS PUBLICAS E GESTAO GOVERN...,CLASSE ESPECIAL,PADRAO V,QUADRO DE PESSOAL DO DISTRITO FEDERAL,00410-00015650/2018-74,"NOS TERMOS DO ARTIGO 3o, INCISOS I, II E III, ...","CONCEDER, aposentadoria voluntaria integral, a...",B-ATO I-ATO O O O O B-NOME_ATO I-NOME_ATO I-NO...


In [57]:
%%time

pipe = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model', xgb.XGBClassifier(
        objective='multi:softprob',
        random_state=42,
        num_class=2
    ))
])    

# pipe.fit(df_train.text, df_train.y);

CPU times: user 988 µs, sys: 0 ns, total: 988 µs
Wall time: 725 µs


In [10]:
%%time
y_pred = pipe.predict(df_test.text)

CPU times: user 3.98 s, sys: 43.9 ms, total: 4.02 s
Wall time: 3.82 s


In [11]:
X_test, y_test = df_test.text, df_test.y
# disp = plot_confusion_matrix(pipe, X_test, y_test,
#                         cmap=plt.cm.Blues,
#                         normalize='true',
#                         xticks_rotation='vertical',
#                         )
f1 = f1_score(y_true=y_test, y_pred=y_pred)
acc = accuracy_score(y_true=y_test, y_pred=y_pred)
print(f'\tf1: {f1:.2f}, acc: {acc:.2f}')
# disp.ax_.set_title(f'left out - {out}')

	f1: 0.98, acc: 1.00


In [61]:
pipe_xgb = pickle.load(open('pipe_xgb.pkl', 'rb'))

In [62]:
y_pred = pipe_xgb.predict(df_ouro.text)

In [67]:
sum(y_pred)/len(y_pred)

0.7272199651770168

In [68]:
pwd

'/home/leomaffei/Desktop/projeto-ia-unb/experiments/members/maffei/aposentadoria-ouro-comparacao'

In [12]:
prf = 'pipe_xgb'
pickle.dump(pipe, open(f'{prf}.pkl', 'wb'))
open(f'{prf}.txt', 'w').write(
    f"notebook de origem: {NOTEBOOK_PATH}\n\n"
    f"`{prf}.pkl` contém um pipeline que contém uma instância\n"
    "treinada do modelo xgboost. A entrada é um vetor de strings\n"
    "e a saída um vetor de inteiros , 1-> verdadeiro 0-> falso\n"
    "(no que diz respeito à presença ou não de atos de aposentadoria).\n"
)

274

In [14]:
df_full_rotulado = pd.concat([df_train, df_test], axis=0)

In [23]:
%%time
X_test_full, y_test_full = (
    df_full_rotulado['text'],
    df_full_rotulado['y']
)
y_pred_full = pipe.predict(X_test_full.values)
f1_full = f1_score(y_true=y_test_full, y_pred=y_pred_full)
acc = accuracy_score(y_true=y_test_full, y_pred=y_pred_full)
print(f'\tf1: {f1:.2f}, acc: {acc:.2f}')


	f1: 0.98, acc: 1.00
CPU times: user 13.9 s, sys: 268 ms, total: 14.2 s
Wall time: 13.3 s


In [25]:
_ = df_full_rotulado[y_pred_full]
_[_.qtd == 0]

,data,num,tipo,text,pnum,x0,y0,x1,y1,y,qtd
5481,2019-03-12,47,NORMAL,REVER os proventos da Aposentadoria de ANGELA ...,42,56.691956,80.326134,404.125793,638.359253,False,0
17278,2018-04-02,62,NORMAL,"ORDEM DE SERVIÇO N° 35, DE 22 DE MARÇO DE 2018...",45,418.109619,421.622986,758.285767,542.122498,False,0


In [27]:
for t in _[_.qtd == 0].text:
    print(t, end='\n'*3)

REVER os proventos da Aposentadoria de ANGELA MARIA CAMPOS MICHELINI matrícula
57.540-2, no Cargo de Professor de Educação Básica, Padrão 23, Etapa II, do Quadro de Pessoal do
Distrito Federal, efetivada por meio da Portaria, nº 49, de 19 de fevereiro de 2003, publicada no
Diário Oficial do Distrito Federal nº 39, de 24 de fevereiro de 2003, a fim de considerá-la inativada
com os proventos integrais nos termos do artigo 18, parágrafo 9º, da Lei Complementar nº 769, de
01 de julho de 2008, com redação dada pelo artigo 291 da Lei Complementar nº 840, de 23 de
dezembro de 2011, com seus efeitos a contar de 01 de julho de 2018. Processo 080 . 0 1 3 11 3 / 2 0 0 2 .
RETIFICAR na Ordem de Serviço de 22 de novembro de 2017 publicada no Diário Oficial do
Distrito Federal, de 29 de novembro de 2017, o ato que aposentou ANGISLANE LUZIA ALBERNAZ
matricula 43.788-3, no Cargo de Professor de Educação Básica, Padrão 25, Etapa IV do Quadro de
Pessoal do Distrito Federal a fim de considerá-lo fundamen

In [28]:
df_full_rotulado.qtd.sum()

3550